In [13]:
import torch
import string
import time
import math
import random
import unicodedata
import torch.nn as nn
import pandas as pd
import torch.optim as optim

from torch.utils.data import DataLoader

In [14]:
class LSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, fc_hidden_dim):
    super().__init__()
    self.hidden_size = hidden_dim
    self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, batch_first=True)
    self.fc1 = nn.Linear(hidden_dim, fc_hidden_dim)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(fc_hidden_dim, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input, hidden):
    out, hidden = self.lstm(input.view(1, 1, -1), hidden)
    out = self.fc1(hidden[0])
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out.view(1, -1), hidden

  def init_hidden(self):
    return (torch.zeros(1, 1, self.hidden_size).cuda(), torch.zeros(1, 1, self.hidden_size).cuda())
    #return (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

In [3]:
inputs = []

df = pd.read_csv('address_dataset.csv')
for index, row in df.iterrows():
  inputs.append((row['text'], row['label']))


random.shuffle(inputs)

235000
('732 24TH AVE E Days Creek PR 72134', 1)


In [4]:
all_letters = string.ascii_letters + " #.,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, 
# thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def letterToIndex(letter):
    return all_letters.find(letter)

def normalize(line):
    if type(line) != str:
      return " "
    line = line.replace(',', '')
    line = line.lower()
    return line

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

def lineToTensor(line):
    line = normalize(line)
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ #.,;'
58
Slusarski
26
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])
t-12
torch.Size([3, 1, 58])


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

n_hidden = 128
n_hidden_2 = 64

lstm = LSTM(n_letters, n_hidden, n_hidden_2)
lstm.to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(lstm.parameters(), lr=0.0001)

def train(batch):
    optimizer.zero_grad()

    for b in batch:
      text, label = b
      text_tensor = lineToTensor(text)
      label_tensor = torch.tensor([[label]], dtype=torch.float).cuda()
      #label_tensor = torch.tensor([[int(label)]], dtype=torch.float)


      hidden = lstm.init_hidden()
      for i in range(text_tensor.size()[0]):
          output, hidden = lstm(text_tensor[i].cuda(), hidden)
          #output, hidden = lstm(text_tensor[i], hidden)

      loss = criterion(output.cuda(), label_tensor)
      #loss = criterion(output, label_tensor)

      loss.backward(retain_graph=True)

    optimizer.step()


    return output, loss.item()

In [6]:
# Keep track of losses for plotting
all_losses = []
batches = 2500

lstm.train()

train_size = int(0.8 * len(inputs))
test_size = len(inputs) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(inputs, [train_size, test_size])

total_loss = 0
train_dataset = list(train_dataset)

for i in range(batches):
    batch = random.sample(train_dataset, 64)
    output, loss = train(batch)
    total_loss += loss

    if i == 0:
      all_losses.append(loss)
      print("Loss: {}".format(loss))

    if i % 250 == 0 and i != 0:
      avg_loss = total_loss / 250
      print("Loss: {}".format(avg_loss))
      all_losses.append(avg_loss)
      total_loss = 0


Loss: 0.6917230486869812
Loss: 0.45231312884157526
Loss: 0.21092578204121673
Loss: 0.09671823677397333
Loss: 0.10218168177560437
Loss: 0.026723304077226204
Loss: 0.04888286434760084
Loss: 0.006745311136190594
Loss: 0.01188792362472077
Loss: 0.007520549940030946
Loss: 0.026105942666538567
Loss: 0.003887763871682239
Loss: 0.04231106172205102
Loss: 0.04422939063763323
Loss: 0.03803845986992263
Loss: 0.01669081492731857
Loss: 0.03042502304759347
Loss: 0.027267929140741217
Loss: 0.01976485121291506
Loss: 0.08951455326959615


In [7]:
def evaluate(line_tensor):
    hidden = lstm.init_hidden()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = lstm(line_tensor[i], hidden)

    return output

def test(test_inputs):

  correct = 0
  with torch.no_grad():
    for test in test_inputs:

      text, label = test
      output = evaluate(lineToTensor(text).cuda())
      #output = evaluate(lineToTensor(text))

      pred = 1 if output.item() >= 0.5 else 0

      if pred == label:
        correct += 1
  
  print("Accuracy: {}".format(correct / len(test_dataset)))


test(test_dataset)

Accuracy: 0.9942553191489362


In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)

NameError: ignored

<Figure size 432x288 with 0 Axes>

In [2]:
def predict(input_line):
    print("{}".format(input_line))
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line).cuda())
        #output = evaluate(lineToTensor(input_line))
        print(output.item())

predict('dfasdfa sfdasfds afas')
predict('Hey hey my my rock n roll will never die')
predict('1600, Washington DC 20500')
predict('Pennsylvania Avenue, Washington 1600')
predict('1600 Pennsylvania Avenue, Washington DC')
predict('1600 Pennsylvania Avenue, Washington DC 20500')

dfasdfa sfdasfds afas


NameError: ignored